# Tutorial

## How to run an image classification model

This tutorial demonstrates how to run a classification model using QAI AppBuilder. Before running the tutorial, refer to the requirements below:

- Install and setup the environment according to [Python setup](python.md). 
- Make sure the dependency libraries from Qualcomm® AI Runtime SDK are ready. To check this, you can see whether there are files in `sample/qai_libs`. You can get more information from [User Guide](user_guide.md#environment-setup) on how to prepare these libraries manually.

## Install Python dependency extension:
Run the following commands to install Python dependency.

In [ ]:
! pip install matplotlib==3.10.3 matplotlib-inline==0.1.7
! pip install notebook==7.4.3 jupyter-client==8.6.3 ipykernel==6.29.5

## Download imagenet_classes.txt
 
Using the following code to download the 'imagenet_classes.txt' file.

In [2]:
import os
import sys
sys.path.append("../samples/python")
import utils.install as install

IMAGENET_CLASSES_URL = "https://raw.githubusercontent.com/pytorch/hub/refs/heads/master/imagenet_classes.txt"
IMAGENET_CLASSES_FILE = "imagenet_classes.txt"
MODEL_NAME = "inception_v3"

workspace = os.getcwd()
model_dir = os.path.abspath(os.path.join(workspace, "inception_v3", "models"))  # The model directory is under docs/inception_v3/models
model_path = os.path.join(model_dir, f"{MODEL_NAME}.bin")
imagenet_classes_path = os.path.join(model_dir, IMAGENET_CLASSES_FILE)

if not os.path.exists(imagenet_classes_path):
    if not install.download_url_pywget(IMAGENET_CLASSES_URL, imagenet_classes_path):
        print("Error while downloading imagenet classes label text.")

## Download model files

For this tutorial, we use pretrained InceptionV3 model. We can download the QNN model from [Qualcomm AI Hub](https://aihub.qualcomm.com) directly. Open [inception_v3](https://aihub.qualcomm.com/compute/models/inception_v3) page and click the button 'Download Model', then select the below options for model downloading: <br>
Choose runtime: Qualcomm® AI Engine Direct. <br>
Choose precision: Both float and w8a8 are acceptable. <br>
Choose device: Snapdragon® X Elite. <br>

Save the downloaded model to path: docs\inception_v3\models\inception_v3.dlc

Refer to [DLC2BIN](../tools/convert/dlc2bin/) to convert inception_v3.dlc to inception_v3.bin.

## Setup QNN environment

After [Python setup](python.md), you should find dynamic libraries files of Qualcomm® AI Runtime SDK under the path: 'samples/qnn_libs'.

Then, you can configure the AppBuilder and load pre-trained model, as shown below.

In [3]:
from qai_appbuilder import (QNNContext, Runtime, LogLevel, ProfilingLevel, PerfProfile, QNNConfig)

qnn_dir = os.path.abspath(os.path.join(workspace, "../samples/qai_libs"))

QNNConfig.Config(qnn_lib_path=qnn_dir, runtime=Runtime.HTP, log_level=LogLevel.WARN, profiling_level=ProfilingLevel.BASIC)

model = QNNContext("inceptionV3", model_path)

### Configurations of initialing SDK

- `qnn_lib_path=qnn_dir`: used to set the SDK library path
- `runtime=Runtime.HTP`: used to set runtimes for model execution on Qualcomm harwdware. (Either `Runtime.HTP` or `Runtime.CPU`)
- `log_level=LogLevel.WARN`: used to set the logging level to WARN.
- `profiling_level=ProfilingLevel.BASIC`: used to set QNN HTP Profiling. For details, please refer to the QNN SDK document.

## Pre-Inference process

Load input image and do preprocessing

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

IMAGE_SIZE = 224

image = Image.open(os.path.join("../samples/python/inception_v3/input.jpg"))
plt.imshow(image)

preprocessor = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
    ]
)
img = preprocessor(image)
img = img.unsqueeze(0).numpy() # Add batch dimension
img = np.transpose(img, (0, 2, 3, 1)) # Convert to NHWC format

## Run inference

To run inference on the preprocessed image, set the performance profile, execute the model, and obtain the output as follows:

- **Set Performance Profile:** Configure the SDK to use the BURST profile for optimal inference speed.
- **Run Inference:** Pass the preprocessed image to the model's `Inference` method.
- **Reset Performance Profile:** Restore the performance profile to its previous state.

The output will contain the model's predictions, which can be further processed to obtain class probabilities and top predictions.

In [8]:
# Set the performance profile to BURST
PerfProfile.SetPerfProfileGlobal(PerfProfile.BURST)

# Run inference
output = model.Inference([[img]])[0]

PerfProfile.RelPerfProfileGlobal()

### Post-processing and Decoding Top-K Predictions

After running inference, the model outputs raw scores (logits). To interpret these results:

1. **Apply Softmax:** Convert logits to probabilities using the softmax function.
2. **Top-K Selection:** Identify the indices of the top K probabilities (e.g., top 5).
3. **Class Decoding:** Map these indices to human-readable class names using the ImageNet class labels.

This process allows you to display the most likely predicted classes for the input image, along with their associated probabilities.

In [9]:
output = torch.from_numpy(output)
probabilities = torch.softmax(output, dim=0)
top5_probabilities, top5_indices = torch.topk(probabilities, 5)

# Read the ImageNet classes
with open(imagenet_classes_path, "r") as f:
    imagenet_classes = [line.strip() for line in f.readlines()]

# Print the top 5 predictions
for i in range(5):
    class_id = top5_indices[i]
    class_name = imagenet_classes[class_id]
    probability = top5_probabilities[i].item()
    print(f"Prediction {i + 1}: {class_name} ({probability:.4f})")


Prediction 1: Samoyed (0.9999)
Prediction 2: Arctic fox (0.0001)
Prediction 3: white wolf (0.0000)
Prediction 4: Eskimo dog (0.0000)
Prediction 5: Pomeranian (0.0000)


## Release the resource

In [15]:
del(model)
model = None